In [ ]:
from utils import install_dependencies, set_seed, print_items_from_dict

In [ ]:
package_names = ["transformers", "nlp", "sentencepiece", "pytorch_lightning==0.9", "rouge_score"]
install_dependencies(package_names)

1. pip install transformers
2. pip install nlp
3. pip install sentencepiece
4. pip install pytorch_lightning==0.9
5. pip install rouge_score



In [ ]:
set_seed(42)

In [ ]:
import json
import argparse
from nlp import load_metric
from model import T5FineTuner
import pytorch_lightning as pl
from transformers import T5Tokenizer
from dataset import wikisql, get_dataset
from callback import LoggingCallback, logger

In [ ]:
dataset = wikisql(type_path='validation',
                  input_length=512,
                  output_length=150,
                  sql2txt=True)
len(dataset)

Using custom data configuration all


Dataset wiki_sql downloaded and prepared to /root/.cache/huggingface/datasets/wiki_sql/all/0.1.0/6c75e26bb08db6bca2dbc201e89b574f053fc25492ef8bbf2614463c14436618. Subsequent calls will reuse this data.


8421

In [ ]:
with open('config.json') as f:
  args_dict = json.load(f)
args = argparse.Namespace(**args_dict)
print_items_from_dict(args_dict)

Items held:
output_dir (t5_wikisql)
model_name (t5-small)
tokenizer_name (t5-small)
max_input_length (512)
max_output_length (150)
freeze_encoder (False)
freeze_embeds (False)
learning_rate (0.0003)
weight_decay (0.0)
adam_epsilon (1e-08)
warmup_steps (0)
train_batch_size (2)
eval_batch_size (2)
num_train_epochs (2)
gradient_accumulation_steps (1)
n_gpu (1)
resume_from_checkpoint (None)
val_check_interval (0.8)
n_val (1000)
n_train (-1)
n_test (-1)
early_stop_callback (False)
fp_16 (False)
opt_level (O1)
max_grad_norm (1.0)
seed (42)
automatic_optimization (False)


In [ ]:
## Define Checkpoint function
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=3
)

## If resuming from checkpoint, add an arg resume_from_checkpoint
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    resume_from_checkpoint=args.resume_from_checkpoint,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    val_check_interval=args.val_check_interval,
    logger=None,
    callbacks=[LoggingCallback()],
)

In [ ]:
model = T5FineTuner(args)

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer = pl.Trainer(**train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model)


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60 M  
Using custom data configuration all


Using custom data configuration all
Using custom data configuration all


Saving latest checkpoint..


1

In [ ]:
batch_temp = model.tokenizer.batch_encode_plus(["SELECT people FROM peoples where age > 10"], max_length=150, 
                                                     padding='max_length', truncation=True, return_tensors="pt")

model.to('cuda')
outs = model.model.generate(
            batch_temp["input_ids"].cuda(),
            attention_mask=batch_temp["attention_mask"].cuda(),
            use_cache=False,
            # decoder_attention_mask=batch['target_mask'].cuda(),
            max_length=150, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0
            # early_stopping=True
        )

dec = [model.tokenizer.decode(ids) for ids in outs]

texts = [model.tokenizer.decode(ids) for ids in batch_temp["input_ids"]]
print("texts: " ,texts)
print("dec: " ,dec)

texts:  ['SELECT people FROM peoples where age > 10</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']
dec:  ['<pad> What is people when age is more than 10?</s>']


In [ ]:
query = "translate Sql to English: SELECT COUNT Params form model where location=HF-Hub </s>"
batch_temp = model.tokenizer.batch_encode_plus(["SELECT COUNT Params form model where location=HF-Hub"], max_length=150,
                                                     padding='max_length', truncation=True, return_tensors="pt")

model.to('cuda')
outs = model.model.generate(
            batch_temp["input_ids"].cuda(),
            attention_mask=batch_temp["attention_mask"].cuda(),
            use_cache=False,
            # decoder_attention_mask=batch['target_mask'].cuda(),
            max_length=150,
            num_beams=2,
            repetition_penalty=2.5,
            length_penalty=1.0
            # early_stopping=True
        )

dec = [model.tokenizer.decode(ids) for ids in outs]

texts = [model.tokenizer.decode(ids) for ids in batch_temp["input_ids"]]
print("texts: " ,texts)
print("dec: " ,dec)

texts:  ['SELECT COUNT Params form model where location=HF-Hub</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']
dec:  ['<pad> Name the total number of params format form model where location isHF-Hub</s>']
